In [2]:

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import cPickle as pickle
import os; import sys; sys.path.append('..')
import gp
import gp.nets as nets

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/nolearn/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [4]:
import numpy as np
import mahotas as mh
import h5py

In [4]:
golds = np.zeros((250,2048,2048), dtype=np.uint64)
rhoanas = np.zeros((250,2048,2048), dtype=np.uint64)
probs = np.zeros((250,2048,2048), dtype=np.uint64)

for z in range(0,250):

    image, prob, mask, gold, rhoana = gp.Util.read_section('/home/d/data/cylinderNEW/', z)
#     masked_image = image.copy()
#     masked_image[mask==0] = 0
    masked_prob = prob.copy()
    masked_prob = 255. - masked_prob
    masked_prob[mask==0] = 0
    golds[z] = gold
    rhoanas[z] = rhoana
    probs[z] = masked_prob

# gp.Util.view(masked_image,color=False)
# gp.Util.view(masked_prob,color=False)

Loading z00000000.png
Loading z00000001.png
Loading z00000002.png
Loading z00000003.png
Loading z00000004.png
Loading z00000005.png
Loading z00000006.png
Loading z00000007.png
Loading z00000008.png
Loading z00000009.png
Loading z00000010.png
Loading z00000011.png
Loading z00000012.png
Loading z00000013.png
Loading z00000014.png
Loading z00000015.png
Loading z00000016.png
Loading z00000017.png
Loading z00000018.png
Loading z00000019.png
Loading z00000020.png
Loading z00000021.png
Loading z00000022.png
Loading z00000023.png
Loading z00000024.png
Loading z00000025.png
Loading z00000026.png
Loading z00000027.png
Loading z00000028.png
Loading z00000029.png
Loading z00000030.png
Loading z00000031.png
Loading z00000032.png
Loading z00000033.png
Loading z00000034.png
Loading z00000035.png
Loading z00000036.png
Loading z00000037.png
Loading z00000038.png
Loading z00000039.png
Loading z00000040.png
Loading z00000041.png
Loading z00000042.png
Loading z00000043.png
Loading z00000044.png
Loading z0

In [5]:
import glob
golds = np.zeros((250,2048,2048), dtype=np.uint32)

path = '/home/d/data/cylinderNEW/'
for z in range(0,250):
    gold = sorted(glob.glob(os.path.join(path, 'gold', '*'+str(z)+'.png')))
    gold = mh.imread(gold[0])
    gold_single = np.zeros((gold.shape[0], gold.shape[1]), dtype=np.uint32)
    gold_single[:, :] = gold[:,:,0]*256*256 + gold[:,:,1]*256 + gold[:,:,2]
    
    golds[z] = gold_single
#
# create gold h5 for neuroproof
#
all_labels = mh.fullhistogram(golds.astype(np.uint64))
all_labels = np.where(all_labels!=0)

transforms = np.zeros((all_labels[0].shape[0],2), dtype=np.uint32)
transforms[:,0] = all_labels[0]
transforms[:,1] = all_labels[0]
print 'generated transforms.'

with h5py.File('/home/d/fp_data/c_gold2.h5', 'w') as hf:
    hf.create_dataset('stack', data=golds)
    hf.create_dataset('transforms', data=transforms)

generated transforms.


In [6]:
import glob
rhoanas = np.zeros((250,2048,2048), dtype=np.uint32)

path = '/home/d/data/cylinderNEW/'
for z in range(0,250):
    rhoana = sorted(glob.glob(os.path.join(path, 'rhoana', '*'+str(z)+'.png')))
    rhoana = mh.imread(rhoana[0])
    rhoana_single = np.zeros((rhoana.shape[0], rhoana.shape[1]), dtype=np.uint32)
    rhoana_single[:, :] = rhoana[:,:,0]*256*256 + rhoana[:,:,1]*256 + rhoana[:,:,2]
    
    rhoanas[z] = rhoana_single
    
#
# create rhoana h5 for neuroproof
#
all_labels = mh.fullhistogram(rhoanas.astype(np.uint64))
all_labels = np.where(all_labels!=0)

transforms = np.zeros((all_labels[0].shape[0],2), dtype=np.uint32)
transforms[:,0] = all_labels[0]
transforms[:,1] = all_labels[0]

with h5py.File('/home/d/fp_data/training/c_rhoana2.h5', 'w') as hf:
    hf.create_dataset('stack', data=rhoanas)
    hf.create_dataset('transforms', data=transforms)

In [6]:
import glob
import tifffile as tif
probs = np.zeros((250,2048,2048), dtype=np.uint8)

path = '/home/d/data/cylinderNEW/'
for z in range(0,250):
    prob = sorted(glob.glob(os.path.join(path, 'prob', '*'+str(z)+'.tif')))
    prob = tif.imread(prob[0])
    mask = sorted(glob.glob(os.path.join(path, 'mask', '*'+str(z)+'.png')))
    mask = mh.imread(mask[0]).astype(np.bool)
    masked_prob = prob
    masked_prob = 255 - masked_prob
    masked_prob[mask==0] = 0
    
    probs[z] = masked_prob
    

#
# create prob h5 for neuroproof
#
p1 = probs
# has to be x,y,z
p1 = p1.swapaxes(0,2)
p1 = p1.swapaxes(0,1)

p2 = p1.copy()
p2 = 255-p2
p_2channels = np.zeros((probs.shape[2],probs.shape[1],probs.shape[0],2), dtype=np.float32)
p_2channels[:,:,:,0] = p1 / 255.
p_2channels[:,:,:,1] = p2 / 255.
with h5py.File('/home/d/fp_data/training/c_prob2.h5', 'w') as hf:
    hf.create_dataset('volume/predictions', data=p_2channels)

In [5]:
sys.path.append('../gp/')